# TPS Feb 2021
Starter Notebook

## Deleverables
1. EDA
    - What's going on?
    - Show me the data...
2. Model
    - Baseline...
    - Simple...
    - Evaluation...
    - Improvement...
3. RAPIDS Bonus
    - Apply RAPIDS ([Starter Notebook](https://www.kaggle.com/tunguz/tps-feb-2021-rapids-starter))
    - Replace pandas with cuDF & sklearn with cuML
    
    
#### Troubleshooting
- [Data](https://www.kaggle.com/c/tabular-playground-series-feb-2021/data)
- [Overview](https://www.kaggle.com/c/tabular-playground-series-feb-2021/overview)
- [RF Starter Notebook](https://www.kaggle.com/warobson/tps-feb-2021-rf-starter)
- [ML repo on GitHub](https://github.com/gumdropsteve/intro_to_machine_learning)
- [Most simple RAPIDS Notebook submission](https://www.kaggle.com/warobson/simple-rapids-live) (Has stuff like `train_test_split()` with cuml..)
    
#### Load Data

In [ ]:
import cudf

train = cudf.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = cudf.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sample_submission = cudf.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

train

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import *
from sklearn.metrics import *

In [ ]:
train.tail(3)

In [ ]:
test.tail(3)

In [ ]:
sample_submission.tail(3)

### Notes
1. To use sklearn with cudf... convert `cudf.Series` / `cudf.DataFrame` `.to_pandas()`..
2. Using cudf is the same as using pandas but `cudf` instead of `pd`..

In [ ]:
type(train)

In [ ]:
type(train.to_pandas())

In [ ]:
train.to_pandas()

In [ ]:
train[['id', 'cat0', 'cat8', 'cont7']].to_pandas()

In [ ]:
type(train.to_pandas())

In [ ]:
y = train.target

y.to_pandas()

In [ ]:
type(y.to_pandas())

# EDA

In [ ]:
train.describe()

In [ ]:
train.shape

In [ ]:
train.describe(exclude ="number")

In [ ]:
# pair plot for all the data 
sns.pairplot(data=train.sample(1000).to_pandas()).set(title="Distribution Of Data");

In [ ]:
# Show the distibution of all data by target
sns.displot(train.to_pandas().target).set(title='Distripution of Target');
plt.show();

In [ ]:
# Show all the data by using BoxPlot 
plt.figure(figsize=(12,8))
sns.boxplot(data=train.to_pandas()).set(title="Box Plot Of Data")

In [ ]:
#Distripution of Target To Check Outliers
plt.figure(figsize=(12,8))
sns.boxplot(train.to_pandas().target).set(title='Distripution of Target (Outliers)');

**Check for Iutliers And Drop it**

In [ ]:
t1=train.loc[train['target'] > 10].to_pandas()
t1

In [ ]:
t2=train.loc[train['target'] < 5 ].to_pandas()
t2

In [ ]:
train.drop(t1.index,inplace=True)

In [ ]:
train.drop(t2.index,inplace=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(train.to_pandas().cont8).set(title='Distripution of Target (Outliers)');

In [ ]:
t3=train.loc[train['cont8'] > 0.9].to_pandas()
t3

In [ ]:
train.drop(t3.index,inplace=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(train.to_pandas().cont8).set(title='Distripution of Target (Outliers)');

In [ ]:
t4=train.loc[train['cont8'] > 0.8].to_pandas()


In [ ]:
t44=train.loc[train['cont8'] < 0.0].to_pandas()


In [ ]:
train.drop(t4.index,inplace=True)

In [ ]:
train.drop(t44.index,inplace=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(train.to_pandas().cont8).set(title='Distripution of Target (Outliers)');

**We Tried To Clean The Data ,But We Didn't For The Lack of Information About The Data**

# Model 

In [ ]:
train = cudf.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
train

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
cat_col = [col for col in train.columns if train[col].dtype=='O']
cat_col

In [ ]:
categorical = train[cat_col]

In [ ]:
categorical_dum = cudf.get_dummies(categorical)
categorical_dum

In [ ]:
df_new = cudf.concat(
    [train,
     categorical_dum],
      axis=1)
df_new

In [ ]:
cat_col = [col for col in df_new.columns if df_new[col].dtype=='O']
df_new = df_new.drop(cat_col, axis = 1)

In [ ]:
df_new

In [ ]:
df_new.drop(['id'],inplace=True,axis=1)

In [ ]:
df_new.columns

In [ ]:
from cuml.preprocessing import train_test_split

X = df_new.drop('target',axis=1)
y = df_new.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
import numpy as np


def baseline_model(n_preds, pred):
    # just predict the average
    return cudf.Series([pred for n in range(n_preds)])

# make baseline preds
baseline_preds = baseline_model(len(y_test), np.mean(y_train))
baseline_preds

In [ ]:
from cuml.metrics import mean_squared_error

mean_squared_error(y_true=y_test,
                   y_pred=baseline_preds,
                   squared=False)

In [ ]:
from cuml.metrics import mean_absolute_error

mean_absolute_error(y_true=y_test,
                   y_pred=baseline_preds)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
# fit the model
regr.fit(X_train.to_pandas(), y_train.to_pandas())

In [ ]:
preds = regr.predict(X_test.to_pandas())

In [ ]:
preds[:-5]

In [ ]:
from cuml.metrics import mean_squared_error

mean_squared_error(y_true=y_test,
                   y_pred=preds,
                   squared=False)

In [ ]:
mean_absolute_error(y_true=y_test,
                   y_pred=preds)

In [ ]:
from cuml.linear_model import LinearRegression

lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
simple_preds = lr.predict(X_test)

simple_preds.tail(5)


In [ ]:
mean_squared_error(y_true=y_test,
                   y_pred=simple_preds,
                   squared=False)

In [ ]:
mean_absolute_error(y_true=y_test,
                   y_pred=simple_preds)

**Optimization**

Random Foriest

In [ ]:
from sklearn.model_selection import GridSearchCV

p_grid = {'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']

}

grid = GridSearchCV(regr, p_grid, cv=7)



In [ ]:
grid.fit(X_train, y_train)


In [ ]:
grid.best_params_

In [ ]:
regr = RandomForestRegressor(random_state=0,
                             max_features='auto', 
                             n_estimators= 200, 
                             max_depth=8, 
 )

In [ ]:
regr.fit(X_train.to_pandas(), y_train.to_pandas())

In [ ]:
preds = regr.predict(X_test.to_pandas())

In [ ]:

mean_squared_error(y_true=y_test,
                   y_pred=preds,
                   squared=False)

In [ ]:
mean_absolute_error(y_true=y_test,
                   y_pred=preds)

Linear Regression

In [ ]:

param_grid = {'fit_intercept':[True, False],
              'normalize':[True,False] }
                      


grid = GridSearchCV(lr, param_grid, cv=7)



In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
lr = LinearRegression(fit_intercept= True, normalize= True)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
sample_preds = lr.predict(X_test)

In [ ]:
mean_squared_error(y_true=y_test,
                   y_pred=sample_preds,
                   squared=False)

In [ ]:
mean_absolute_error(y_true=y_test,
                   y_pred=sample_preds)

In [ ]:
test.isnull().sum()

In [ ]:
test.drop('id',axis=1,inplace=True)

In [ ]:
cat_col = [col for col in test.columns if train[col].dtype=='O']
cat_col

In [ ]:
categorical = test[cat_col]

In [ ]:
categorical_dum = cudf.get_dummies(categorical)
categorical_dum

In [ ]:
new_test= cudf.concat(
    [test,
     categorical_dum],
      axis=1)
new_test

In [ ]:
cat_col = [col for col in new_test.columns if new_test[col].dtype=='O']
cat_col

In [ ]:
new_test= new_test.drop(cat_col, axis = 1)

In [ ]:
new_test

In [ ]:
new_test['cat6_G'] = 0

In [ ]:
test_preds = lr.predict(new_test)
test_preds

In [ ]:
simple_preds = lr.predict(new_test)

# save results & submit
sample_submission['target'] = simple_preds

sample_submission.to_csv('submission.csv', index=False)